# Describe

In [1]:
# nuclio: start-code

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from mlrun.execution import MLClientCtx
from mlrun.datastore import DataItem
from mlrun.artifacts import PlotArtifact, TableArtifact

from sklearn.preprocessing import StandardScaler
from yellowbrick import ClassBalance

from typing import IO, AnyStr, Union, List, Optional

pd.set_option("display.float_format", lambda x: "%.2f" % x)

def _gcf_clear(plt):
    plt.cla()
    plt.clf()
    plt.close() 

def describe(
    context: MLClientCtx,
    table: Union[DataItem, str],
    label_column: str,
    class_labels: List[str] = None,
    key: str = "table-summary",
    plot_hist: bool = True,
    plots_dest: str = 'plots'
) -> None:
    """Summarize a table

    TODO: merge with dask version

    :param context:         the function context
    :param table:           pandas dataframe
    :param label_column:    ground truth column label
    :param class_labels:    label for each class in tables and plots
    :param key:             key of table summary in artifact store
    :param plot_hist:       (True) set this to False for large tables
    :param plots_dest:      destination folder of summary plots (relative to artifact_path)
    """
    base_path = context.artifact_path
    os.makedirs(base_path, exist_ok=True)
    os.makedirs(os.path.join(base_path, plots_dest), exist_ok=True)
    
    table = pd.read_parquet(str(table))
    header = table.columns.values

    _gcf_clear(plt)
    try:
        snsplt = sns.pairplot(table, hue=label_column)
        snsplt.savefig(os.path.join(base_path, f"{plots_dest}/hist.png"))
        context.log_artifact(PlotArtifact("histograms",  body=plt.gcf()), local_path=f"{plots_dest}/hist.html")
    except Exception as e:
        context.logger.warning(f'While trying to run sns.pairplot encountered the following error: {e}')

    sumtbl = table.describe()
    sumtbl = sumtbl.append(len(table.index)-table.count(), ignore_index=True)
    sumtbl.insert(0, "metric", ["count", "mean", "std", "min","25%", "50%", "75%", "max", "nans"])
    
    sumtbl.to_csv(os.path.join(base_path, key+".csv"), index=False)
    context.log_artifact(key, local_path=key+".csv")

    _gcf_clear(plt)
    
    labels = table.pop(label_column)
    class_balance_model = ClassBalance(labels=class_labels)
    class_balance_model.fit(labels)
    
    scale_pos_weight = class_balance_model.support_[0]/class_balance_model.support_[1]
    context.log_artifact("scale_pos_weight", f"{scale_pos_weight:0.2f}")

    class_balance_model.show(outpath=os.path.join(base_path, f"{plots_dest}/imbalance.png"))
    context.log_artifact(PlotArtifact("imbalance", body=plt.gcf()), local_path=f"{plots_dest}/imbalance.html")
    
    _gcf_clear(plt)
    tblcorr = table.corr()
    ax = plt.axes()
    sns.heatmap(tblcorr, ax=ax, annot=False, cmap=plt.cm.Reds)
    ax.set_title("features correlation")
    plt.savefig(os.path.join(base_path, f"{plots_dest}/corr.png"))
    context.log_artifact(PlotArtifact("correlation",  body=plt.gcf()), local_path=f"{plots_dest}/corr.html")
    
   
    _gcf_clear(plt)

# nuclio: end-code

/User/.conda/envs/stable/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
